In [ ]:
import os
import cv2
from deepface import DeepFace
import FaceNormalizationUtils as faceutils
import numpy as np
import FaceDetectors


In [15]:
# Initialize normalization
normalizatorHS = faceutils.Normalization()

# Initialize face detector
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX


# Webcam connection
cap = cv2.VideoCapture(0)

# Set camera resolution
cap.set(3, 640)
cap.set(4, 480)

while True:
    # Get frame
    ret, frame = cap.read()

    # For HS normalization
    B, G, R = cv2.split(frame)
    result = frame.copy()
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])


    if values is not None:

        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(result, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face_roi = frame[y:y+h, x:x+w]
            obj = DeepFace.analyze(img_path = face_roi, enforce_detection=False, actions =['race'])
            race_detected = obj[0]["dominant_race"]
            if race_detected == "asian":
                background = cv2.imread('media/wano.jpg')

            elif race_detected == "black":
                background = cv2.imread('media/savannah.jpg')

            elif race_detected == "white":
                background = cv2.imread('media/de_blanco.jpg')

            elif race_detected == "indian":
                background = cv2.imread('media/Taj_Mahal.jpg')

            elif race_detected == "middle eastern":
                background = cv2.imread('media/mezquita.jpg')

            elif race_detected == "latino hispanic":
                background = cv2.imread('media/brazil.jpg')

            background = cv2.resize(background, (int(cap.get(3)), int(cap.get(4))))
            result = background.copy()
            result[y:y+h, x:x+w] = cv2.addWeighted(face_roi, 0.9, result[y:y+h, x:x+w], 0.1, 0)
    
    cv2.imshow('Result', result)

    # Press 'Esc' to finish
    key = cv2.waitKey(40)
    if key == 27:  # Esc
        break

# Close windows and release the camera
cap.release()
cv2.destroyAllWindows()


Action: race: 100%|██████████| 1/1 [00:00<00:00,  4.20it/s]


In [ ]:

cap = cv2.VideoCapture(0)
background = cv2.imread('media/africa.jpg')
background = cv2.resize(background, (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    result = cv2.addWeighted(frame, 0.8, background, 0.7, 0)

    cv2.imshow('Result', result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [14]:
folder = './media'

print(folder)

for file_name in os.listdir(folder):
    # Asume imágenes en formato png o jpg
    if file_name.endswith('.png') or file_name.endswith('.jpg'):
        # Procesa la imagen que asume hay cara, no fuerza la detección
        obj = DeepFace.analyze(img_path = os.path.join(folder, file_name), enforce_detection=False, actions =['race'])
        print(file_name)
        print(obj[0]['race'])
       
    

./media


Action: race: 100%|██████████| 1/1 [00:03<00:00,  3.44s/it]


brazil.jpg
{'asian': 0.18457091646268964, 'indian': 1.1787673458456993, 'black': 0.13584352564066648, 'white': 58.97362232208252, 'middle eastern': 29.39012348651886, 'latino hispanic': 10.13706848025322}


Action: race: 100%|██████████| 1/1 [00:00<00:00,  1.87it/s]


de_blanco.jpg
{'asian': 11.085090041160583, 'indian': 3.897593542933464, 'black': 68.75017881393433, 'white': 9.162162989377975, 'middle eastern': 2.2778326645493507, 'latino hispanic': 4.827144369482994}


...

savannah.jpg
{'asian': 4.079829399575555, 'indian': 3.556512508184463, 'black': 1.0572596151983509, 'white': 41.66265378371798, 'middle eastern': 28.72076801781464, 'latino hispanic': 20.9229779793607}


Action: race: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


Taj_Mahal.jpg
{'asian': 1.2094603851437569, 'indian': 1.6810433939099312, 'black': 0.17503321869298816, 'white': 55.784791707992554, 'middle eastern': 33.10583233833313, 'latino hispanic': 8.043842017650604}


Action: race: 100%|██████████| 1/1 [00:00<00:00,  2.52it/s]

wano.jpg
{'asian': 0.1189013128168881, 'indian': 0.4676187876611948, 'black': 0.005580256402026862, 'white': 68.56342554092407, 'middle eastern': 21.805012226104736, 'latino hispanic': 9.039466828107834}
